In [ ]:
from os.path import join

import pystac_client
from shapely.geometry import mapping

from rastervision.pipeline.file_system.utils import json_to_file, file_to_json
from rastervision.aws_sagemaker import AWSSageMakerRunner
from rastervision.core.box import Box
from rastervision.core.data import (
    RasterioSourceConfig, SceneConfig, SemanticSegmentationLabelStoreConfig,
    XarraySourceConfig, STACItemConfig)
from rastervision.core.predictor import Predictor2

# Get a Sentinel-2 scene from a STAC API

Define AOI:

In [2]:
# Paris, France
bbox = Box(ymin=48.8155755, xmin=2.224122, ymax=48.902156, xmax=2.4697602)
bbox_geometry = mapping(bbox.to_shapely().oriented_envelope)

Query STAC API:

In [3]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

items = catalog.search(
    intersects=bbox_geometry,
    collections=["sentinel-2-l2a"],
    datetime="2023-06-01/2023-06-10",
).get_all_items()
len(items)

3

Pick a specific STAC item and serialize it to a file on S3:

In [10]:
item = items[0]
item_uri = join('s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/', 'stac-item.json')
json_to_file(item.to_dict(), item_uri)

# Configure Raster Vision

Define a Raster Vision [`SceneConfig`](https://docs.rastervision.io/en/stable/api_reference/_generated/rastervision.core.data.scene_config.SceneConfig.html#sceneconfig), which specifies how the raster is to be read and how the predictions are to be written:

In [3]:
raster_source_config = XarraySourceConfig(
    stac=STACItemConfig(
        uri=item_uri, 
        assets=[
        'coastal', # B01
        'blue', # B02
        'green', # B03
    ]),
    bbox_map_coords=tuple(bbox),
)

pred_uri = join('s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/')
label_store_config = SemanticSegmentationLabelStoreConfig(uri=pred_uri)

scene_config = SceneConfig(
    id='test_scene',
    raster_source=raster_source_config,
    label_store=label_store_config,
)

Serialize the `SceneConfig` save it to a file on S3:

In [4]:
scene_config_uri = join('s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/', 'scene-config.json')
json_to_file(scene_config.dict(), scene_config_uri)

Inspect S3 directory:

In [5]:
! aws s3 ls "s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/"

2023-11-05 20:49:23        696 scene-config.json
2023-11-05 20:49:21      23020 stac-item.json


Specify a URI for the Raster Vision model-bundle:

In [ ]:
# This particular model isn't actually trained on Sentinel-2 imagery and so
# won't produce meaningful outputs. It is used here for demo purposes.
model_bundle_uri = 'https://s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip'

# Run the prediction on AWS SageMaker

Define CLI command to run the prediction:

In [6]:
cmd = ['rastervision', 'predict_scene', model_bundle_uri, scene_config_uri]
' '.join(cmd)

'rastervision predict_scene https://s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/scene-config.json'

Run the command on AWS SageMaker:

In [8]:
runner = AWSSageMakerRunner()
runner.run_command(
    cmd=cmd,
    use_gpu=True,
    job_name='stac-pred',
)

INFO:botocore.tokens:Loading cached SSO token for rv
INFO:sagemaker:Creating processing-job with name stac-pred-2023-11-05-20-49-55-843


.........................................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
2023-11-05 20:56:53:rastervision.pipeline.file_system.utils: INFO - Downloading https://s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip to /opt/data/tmp/cache/http/s3.amazonaws.com/azavea-research-public-data/raster-vision/examples/model-zoo-0.21/isprs-potsdam-ss/model-bundle.zip...
#015 80%|███████▉  | 166M/209M [00:05<00:01, 34.5MB/s]#015 87%|████████▋ | 183M/209M [00:05<00:00, 31.9MB/s]#015 93%|█████████▎| 195M/209M [00:06<00:00, 30.4MB/s]#015 99%|█████████▉| 208M/209M [00:07<00:00, 29.4MB/s]#015100%|██████████| 209M/209M [00:07<00:00, 31.3MB/s]
2023-11-05 20:57:00:rastervision.pytorch_learner.learner: INFO - Loading learner from bundle /opt/data/tmp/tmpg4la3o4i/bundle/model-bundle

Inspect the S3 directory again. Note the new `labels.tif` file containing predictions.

In [9]:
! aws s3 ls "s3://raster-vision-ahassan/rvtest/stac_pred_sagemaker/20231105/"

2023-11-05 20:57:12    1714462 labels.tif
2023-11-05 20:49:23        696 scene-config.json
2023-11-05 20:49:21      23020 stac-item.json
